# Clase 23: Compilación, Paralelismo y Computación Distribuida

**MDS7202: Laboratorio de Programación Científica para Ciencia de Datos**

**Profesor: Pablo Badilla**

## Objetivos de la clase:

- Aprender a optimizar código a través de `JIT`.
- Comprender el paralelismo de tareas.
- Aprender a paralelizar tareas por medio de funciones en `Joblib`
- Comprender la idea general de computación distribuida.
- Analizar las opciones para computación distribuida: `Dask`.

## Motivación

El flujo de trabajo en ciencia de datos consta de **numerosas rutinas de carga, procesamiento y visualización**. Lo ideal es que diseñemos estas rutinas de la forma más optima posible con el fin de reducir recursos, tiempos de carga utilizados y sus costos asociados.


## Lenguajes de Programación

El lenguaje de máquina es el conjunto de instrucciones que el hardware es capaz de interpretar y procesar.
A través de estas instrucciones podemos lograr que nuestro procesador ejecute distintos tipos de acciones muy básicas. 
Este conjuntos de lenguajes es comunmente conocido como *lenguaje de bajo nivel*

<center>
<img src='https://raw.githubusercontent.com/MDS7202/MDS7202/main/recursos/2023-01/23_compilacion_y_paralelismo/codigo_maquina.png' width=400 />
<center/>

<center>Por suerte no tenemos que si quiera pensar en esto...</center>
    
<center> 
    Fuente: <a href='https://en.wikipedia.org/wiki/Machine_code#/media/File:W65C816S_Machine_Code_Monitor.jpeg'>Wikipedia </a>
</center>
    
    
    


---

## Lenguajes Compilados vs Intepretados

Existen dos enfoques principales para convertir un código de lenguaje de alto nivel a uno de bajo nivel: que el lenguaje sea **Compilado** o **Interpretado**.


<br>
<center>
<img src='https://raw.githubusercontent.com/MDS7202/MDS7202/main/recursos/2023-01/23_compilacion_y_paralelismo/tipos_lenguajes.png' width=800 />
</center>


---

## Computación de alto Rendimiento con Python

Python es utilizado transversalmente, ya sea en la industria o en la academia. Dentro de sus cualidades se encuentra la portabilidad de código, sintaxis intuitiva, disponibilidad de herramientas y documentación. Sin embargo, al ser un lenguaje interpretado se pierden ciertas características intrínsicas de los lenguajes de bajo nivel como C, C++ y Fortran.

En esta y la próxima clase estudiaremos distintas herramientas para mejorar el rendimiento del interprete, como el uso eficiente de objetos base y la aplicación de técnicas de paralelismo y compilación utilizando tanto librerías nativas, como desarrolladas por terceros. 


> **Pregunta ❓:** ¿Será conveniente programar siempre pensando crear código óptimo?

---

## Optimización del Código

 Como directriz general, se recomienda llevar el proceso de desarrollo en dos etapas:
 
1. La primera consiste en **generar código correcto, comprensibles y mantenibles**, evitando la sobre-optimización prematura de código. 

2. Como segunda etapa, se recomienda comenzar con los procesos de **optimización de código**. Esto pues, las herramientas que permiten mejorar los aspectos computacionales, interfieren en la sencillez del código, entorpeciendo los procesos de depuración y mantención. 

Una vez que las rutinas están implementadas de manera correcta, la mejor manera de enfocar los esfuerzos, pasa por **perfilar** (*profiling*) el código. Esto consiste en encontrar las zonas de código criticas en cuanto a carga computacional. La manera más directa de encontrar estas zonas, es por medio del uso de contadores de tiempo o *timers*.



### Medición del Tiempo de Ejecución ⏰


El tiempo de ejecución es el tiempo tomado por algun segemento de código, función en completar su ejecución.

En Python, la forma más sencilla de medir el tiempo de ejecución es a través de la librería `time`. El ejemplo siguiente muestra como utilizarla.

In [ ]:
import time
from math import cos, sin

Definimos un rango de datos a operar 

In [ ]:
x = [0.1 * i for i in range(100000)]

x[0:10]  # veamos los datos

Luego definimos la función que mediremos. Esta simplemente calcula $(\sin(val) + \cos(val)^2)^{1/3}$ y luego retorna su valor.

In [ ]:
def func_1(val):
    return (sin(val) + cos(val) ** 2) ** (1 / 9)

Ahora, estudiamos el tiempo de ejecución por medio de la función `process_time`.

In [ ]:
# tiempo inicial
t0 = time.process_time()

for i in x:
    func_1(i)

# tiempo final
t1 = time.process_time()

# el tiempo transcurrido es simplemente el delta entre t1 y t0
print("Tiempo transcurrido", t1 - t0)

> **Pregunta ❓:** ¿Si ejecutamos nuevamente la celda anterior, obtendremos los mismos tiempos? ¿Existirá alguna forma más consistente de medir el tiempo de ejecución del código?

---

### `timeit`

En algunas ocasiones se desea medir el tiempo de ejecución para tareas sencillas, la librería estándar de Python provee el módulo `timeit`. En la práctica, una llamada de `timeit` ejecuta por defecto 10.000.000 el código (variable según cuánto se demore el proceso) y repite 7 veces el experimento. Lluego reporta el tiempo de ejecución promedio.

Este puede ser utilizado directamente en la consola interactiva IPython o en notebooks de Jupyter por medio del comando mágico `%timeit` para el caso de una linea de código y `%%timeit` para medir toda la celda. 

Documentación de `%timeit`: [Timeit Magic en la documentación de Ipython](https://ipython.readthedocs.io/en/stable/interactive/magics.html#magic-timeit)



**Ejemplo**


Medimos la eficiencia de la implementación original de python de `cos`.

In [ ]:
%timeit cos(0.5)

Y lo comparamos con el tiempo de ejecución promedio para la función coseno de `numpy`.

In [ ]:
import numpy as np

In [ ]:
%timeit np.cos(0.5)

In [ ]:
%timeit func_1(100)

> **Pregunta ❓:** ¿Se podrá medir el tiempo que toma cada instrucción por separado?

---

## Compiladores

<center>
<img src='https://raw.githubusercontent.com/MDS7202/MDS7202/main/recursos/2023-01/23_compilacion_y_paralelismo/numba.png' width=600/>
</center>

Un proyecto interesante la librería **`Numba`** la cual está enfocada en **analizar y compilar funciones de Python**. Compiladores como Numba, diseñados para compilar código en ejecución (y no previo a la ejecución) se denomina compiladores **JIT** (just in time). 

Numba permite compilar funciones individuales de Python usado una *máquina virtual de bajo nivel* o LLVM por sus siglas en inglés (LLVM es un conjunto de herramientas pensadas para escribir compiladores).

Por medio de LLVM Numba inspecciona funciones de Python y las compila utilizando una capa de representación intermedia similar a código *assembly*. La potencia de esta inspección radica en la inferencia de tipos de datos generando una versiones compiladas con tipos de datos estáticos.

Numba se basa principalmente en el decorador `@jit` con el cual se definen las funciones a compilar.

**Ejemplo: Calcular el valor de $\pi$ usando Montecarlo**


Idea: 

<div align='center'>
<img src='https://raw.githubusercontent.com/MDS7202/MDS7202/main/recursos/2023-01/23_compilacion_y_paralelismo/montecarlo.png' width=300 />
<div/>
    
$$\frac{\text{area círculo}}{\text{area cuadrado}} = \frac{\pi r^2}{(2r)^2} $$

$$ 4* \frac{\text{area círculo}}{\text{area cuadrado}} = \pi $$


Y después simulamos que lanzamos puntos al azar a nuestra figura y contamos: 

$$ 4* \frac{\text{puntos en el circulo}}{\text{puntos en el cuadrado}} = \pi $$





Para comprobar el aumento de rendimiento de la compilación, usaremos 3 implementaciones distintas:
    
    1. Python.
    2. Numpy.
    3. Python con Numba.

### $\pi$ con Montecarlo en `Python`

In [ ]:
import random


def monte_carlo_pi_python(nsamples):
    acc = 0
    for i in range(nsamples):
        x = random.random()
        y = random.random()

        if (x**2 + y**2) < 1.0:
            acc += 1

    return 4.0 * acc / nsamples

In [ ]:
monte_carlo_pi_python(10000000)

In [ ]:
%timeit monte_carlo_pi_python(100000)

### $\pi$ con Montecarlo en `Numpy`

In [ ]:
import numpy as np


def monte_carlo_pi_numpy(nsamples):
    acc = 0
    x = np.random.rand(nsamples)
    y = np.random.rand(nsamples)

    op = x**2 + y**2
    dentro_circulo = op[op < 1.0]

    return 4.0 * np.count_nonzero(dentro_circulo) / nsamples

In [ ]:
monte_carlo_pi_numpy(100000)

In [ ]:
%timeit monte_carlo_pi_numpy(100000)

### $\pi$ con Montecarlo en `Numba`

Y ahora probamos con una función compilada usando el decorador `@jit`.

In [ ]:
import random

from numba import jit


@jit(nopython=True)
def monte_carlo_pi_numba(nsamples):
    acc = 0
    for i in range(nsamples):
        x = random.random()
        y = random.random()

        if (x**2 + y**2) < 1.0:
            acc += 1

    return 4.0 * acc / nsamples

In [ ]:
monte_carlo_pi_numba(100000)

In [ ]:
%timeit monte_carlo_pi_numba(100000)

### Numba y Numpy

`Numba` también está diseñado para funcionar en conjunto con `numpy`

In [ ]:
@jit(nopython=True)
def monte_carlo_pi_numpy_numba(nsamples):
    acc = 0
    x = np.random.rand(nsamples)
    y = np.random.rand(nsamples)

    op = x**2 + y**2
    dentro_circulo = op[op < 1.0]

    return 4.0 * np.count_nonzero(dentro_circulo) / nsamples

In [ ]:
monte_carlo_pi_numpy_numba(100000)

In [ ]:
%timeit monte_carlo_pi_numpy_numba(100000)

### Importante: `Numba` solo compila código de Python y `Numpy`

Está en general diseñado para optimizar tareas matemáticas y con ciclos.
No entiende librerías más complejas como `pandas` por ejemplo.





In [ ]:
import pandas as pd

x = {"a": [1, 2, 3], "b": [20, 30, 40]}

In [ ]:
def use_pandas(a):  # Function will not benefit from Numba jit
    df = pd.DataFrame.from_dict(a)  # Numba doesn't know about pd.DataFrame
    df += 1  # Numba doesn't understand what this is
    return df.cov()  # or this!

In [ ]:
%timeit use_pandas(x)

In [ ]:
@jit(nopython=True)
def use_pandas(a):  # Function will not benefit from Numba jit
    df = pd.DataFrame.from_dict(a)  # Numba doesn't know about pd.DataFrame
    df += 1  # Numba doesn't understand what this is
    return df.cov()  # or this!

In [ ]:
use_pandas(x)

In [ ]:
%timeit use_pandas(x)

---

## Paralelismo

El paralelismo se basa en el uso de múltiples unidades de computo de manera simulánea, con el el fin de mejorar la eficiencia en rutinas de código. La idea principal consite en enfrentar un problema de programación, dividiendolo en subunidades independientes y utilizar los núcleos disponibles de la máquina para resolver tales subunidades en paralelo.

<img src='https://raw.githubusercontent.com/MDS7202/MDS7202/main/recursos/2023-01/23_compilacion_y_paralelismo/paralelo_vs_secuencial.jpeg'/>
<center>
Fuente: 
<a href='https://towardsdatascience.com/an-intro-to-parallel-computing-with-ray-d8503629485'>https://towardsdatascience.com/an-intro-to-parallel-computing-with-ray-d8503629485</a>
    
</center>



---

### Problemas Data Parallel

Los problemas **Data Parallel** son aquellas en las que se le aplica una función particular sobre todos los datos (por ejemplo, multiplicar una matriz por un escalar).


En este tipo de problema paralelizable, es importante que la función es exactamente la misma y que el calculo de esta es independiente de todas las otras funciones. Por lo mismo, estas tareas también son denominadas **perfectamente paralelizables**. 

Las operaciones elemento por elemento sobre arreglos poseen esta propiedad. 


<div align='center'>
<img src='https://raw.githubusercontent.com/MDS7202/MDS7202/main/recursos/2023-01/23_compilacion_y_paralelismo/cpu_gpu.jpg' width=500 />
</div>

<div align='center'>
    Fuente: 
<a href='https://www.nvidia.com/es-la/drivers/what-is-gpu-computing/'>Nvidia.</a>
</div>

  
Imaginense la cantidad de operaciones simples que una GPU puede lograr hacer en paralelo. Por ejemplo, sumar una matriz con otra elemento a elemento.
  



---

### Problemas Task Parallel

Los problemas task parallel son aquellos que ejecutan varias tareas distintas en distintos hilos/procesos sobre distintos procesadores.

<center>
<img src='https://raw.githubusercontent.com/MDS7202/MDS7202/main/recursos/2023-01/23_compilacion_y_paralelismo/paralelismo_memoria.png' width=500 />
</center>

<center>
Fuente:    
<a href='https://manningbooks.medium.com/explaining-mapreduce-with-ducks-f643c78e0b40'>https://manningbooks.medium.com/explaining-mapreduce-with-ducks-f643c78e0b40</a>
</center>

<br>


Por lo general, este tipo de tasks no son completamente independientes y necesitan compartir información. En estos casos, se debe tener en cuenta que la comunicación entre subunidades y los datos compartidos **quitan eficiencia** al problema que se resuelve, pues se incurre en *costos de comunicación*. 

La comunicación entre procesos es inherentemente costosa y puede llevar fallas de correctitud . Por lo general, se enfrenta el problema de costo de comunicación y correctud del manejo de memoria por medio de sistemas que se comunican por medio de **threads/hilos con memoria compartida** y **procesos con memoria distribuida**.

---

### Hilos de Procesamiento o Threads

En el caso de memoria compartida, las subunidades involucradas en el programa tienen acceso a un espacio común de memoria, este por lo general es de acceso rápido. 

Si bien esto solventa el problema de velocidad de comunicación, el problema de correctitud sigue latente, por lo que se hace necesario utilizar técnicas de **sincronización**. 

La manera usual en la que se implementan procesos de memoria compartida es por medio de **threads o hilos de ejecución**. Estos consisten en subtareas originadas de un proceso en particular y que comparten recursos. 


<center>
<img src='https://raw.githubusercontent.com/MDS7202/MDS7202/main/recursos/2023-01/23_compilacion_y_paralelismo/threads.jpg' width=500/>
</center>

<center>
Fuente:
<a href='https://www.cs.uic.edu/~jbell/CourseNotes/OperatingSystems/4_Threads.html'> https://www.cs.uic.edu/~jbell/CourseNotes/OperatingSystems/4_Threads.html </a>
</center>

---

### Procesos

Por otra parte, el concepto de memoria distribuida concibe cada subunidad como un proceso completamente separado del resto con su propio espacio de memoria asociado. En este caso, la comunicación entre procesos se debe manejar de manera explicita y es más costosa que en el caso de memoria compartida, sin embargo, se reduce el riesgo de generar errores en el manejo de memoria. 

Este tipo de paralelismos puede ser observadas en los distintos procesos que ejecuta nuestro computador.

<center>
<img src='https://raw.githubusercontent.com/MDS7202/MDS7202/main/recursos/2023-01/23_compilacion_y_paralelismo/thread_process.png' width=500/>
</center>


<center>
    Fuente:
    <a href='https://www.javamex.com/tutorials/threads/how_threads_work.shtml'>https://www.javamex.com/tutorials/threads/how_threads_work.shtml</a>
</center>

    
<br>
<br>

> **Pregunta ❓**: ¿Qué aplicación de data science podría beneficiarse de la aplicación de procesos paralelos?

---

### Threads y Procesos en Python

Python puede manejar threads pero dado el diseño de su interprete, por defecto, se puede ejecutar solo una tarea a la vez, esto se conoce como **GIL** (Global Interpreter Lock). GIL provoca que cada vez que un hilo ejecute una orden de Python, se genere un bloqueo que solo será liberado una vez la ejecución del hilo termine.

> **Esto hace que los hilos solo puedan ser ejecutados de manera secuencial.**

Es decir, Python no puede ejecutar 2 o más hilos de ejecución al mismo tiempo usando más de un procesador.


Aunque GIL evita la ejecución de hilos usando múltiples procesadores en paralelo, es posible utilizar procesos mediante algunas librerías. La principal es `multiprocessing`

Multiprocessing ofrece una interfaz sencilla que incluye múltiples herramientas para manejar sincronozación y ejecución de tareas. Es posible importar esta librería de manera estándar. 

```python
import multiprocessing
```

Es posible crear procesos independientes por medio la clase `Process`, para ello basta extender el método `__init__` para inicializar los datos a procesar y generar el método `run` sobre el cual se ejecuta el proceso.

**Ejemplo**
 
Se genera un proceso independiente utilizando la clase `Process`

In [ ]:
import time
from multiprocessing import Process


class Proceso_independiente(Process):
    def __init__(self, num):
        super().__init__()
        self.num = num

    def run(self):
        print("Mi número:", self.num, "\nMe voy a dormir 10s 💤😴💤")
        time.sleep(10)
        print("Desperté 😃")

Para utilizar el proceso se instancia un objeto de la clase `Proceso_ind` y se llama el método `.start()` 

In [ ]:
proc = Proceso_independiente(5)
proc.start()

In [ ]:
proc = Proceso_independiente(10)
proc.start()

In [ ]:
print("¿¿¿🤨??? Me puedo ejecutar sin esperar a que la celda anterior termine")

**Obs**:En el ejemplo anterior, no fue necesario utilizar el metodo anulado `.run()`, este es llamado por `.start()` de manera interna.

En el caso en que se requiera esperar la finalización de un conjunto de tareas paralelas para luego recopilar resultados, es posible utilizar el método `.join()`.

In [ ]:
proc = Proceso_independiente(5)
proc.start()
proc.join()

print("Aquí tuve que esperar 😔")

Con la construcción actual, es posible levantar tantos procesos como se requiera, en esta caso se levantan 3 procesos.

In [ ]:
import time
from multiprocessing import Process


class Proceso_independiente(Process):
    def __init__(self, num):
        super().__init__()
        self.num = num

    def run(self):
        print(f"Me voy a dormir 10s ({self.num})💤😴💤\n")
        time.sleep(10)
        print(f"Desperté ({self.num})😃\n")

In [ ]:
# Se definen los 3 procesos
procesos = (
    Proceso_independiente(1),
    Proceso_independiente(2),
    Proceso_independiente(3),
)

# Se mide el tiempo de ejecucion
start = time.time()

# Iniciar todos los procesos
for p in procesos:
    p.start()

# Esperar a que terminen todos los procesos
for p in procesos:
    p.join()

end = time.time()


print("Tiempo de ejecución: ", end - start)

Estos tres procesos corren de manera paralela, pues su tiempo de ejecución total es aproximado al tiempo de ejecución individual. 

Es necesario comprender que el orden de ejecución de procesos paralelos no es necesariamente ordenando y predecible pues depende de cómo el sistema operativo asigne los recursos. 

### Memoria Compartida y Dataraces

Un data race es una situación que ocurre cuando uno o más hilos acceden concurrentemente a una posición de memoria o variable, al menos uno está escribiendo y al menos uno no está sincronizado con los otros hilos.

<div align='center'>
<img src='https://raw.githubusercontent.com/MDS7202/MDS7202/main/recursos/2023-01/23_compilacion_y_paralelismo/datarace_1.png' width=400/>
</div>

<div align='center'>
    Ejecución secuencial en memoria compartida por threads.
    Fuente: <a href='https://en.wikipedia.org/wiki/Race_condition'>Wikipedia</a>
</div>

<br>
<br>

<div align='center'>
<img src='https://raw.githubusercontent.com/MDS7202/MDS7202/main/recursos/2023-01/23_compilacion_y_paralelismo/datarace_2.png' width=400/>
</div>

<div align='center'>
    Ejecución paralela en memoria compartida por threads.
    Fuente: <a href='https://en.wikipedia.org/wiki/Race_condition'>Wikipedia</a>
</div>

<br>

**La solución es tener mecanismos de sincronización** de hilos. 



### Ejemplo en `multiprocessing`


El comportamiento predeterminado de `multiprocessing` es generar procesos con memoria independiente, sin embargo, permite definir ciertas variables en memoria compartida. Para definir una variable en memoria compartida se utiliza la clase `Value`, a esta clase se le entrega un tipo de dato que puede ser `i` para entero, `f` para flotante, `d` para doble precisión entre otros. 


In [ ]:
from multiprocessing import Value

comp_var = Value("d")

Al utilizar variables en memoria compartida se deben tener en cuenta los procesos que acceden a ella, manejando la *concurrencia*, es decir, si los procesos pueden acceder a dichas variables de manera simultanea u ordenada. Por lo general en la actualización de valores unidimensionales se debe tener en cuenta la concurrencia bloqueando el acceso simultaneo. En arreglos se puede permitir tal manipulación siempre que los computos sean independientes. 

Para bloquear el acceso a una variable compartida se hace uso de la clase `Lock`.

In [ ]:
from multiprocessing import Lock

lock = Lock()

A continuación se genera una rutina que accede a una variable de memoria compartida

In [ ]:
from multiprocessing import Process, Value


class Process_shared(Process):
    def __init__(self, var, n=10000):
        super().__init__()
        self.var = var
        self.n = n

    def run(self):
        for i in range(self.n):
            self.var.value += 1

El proceso asociado toma un valor y le añade 1 hasta `n = 10000` veces por proceso. Se crea el valor inicial y se inicializan 3 procesos

In [ ]:
def test():
    var = Value("i")
    var.value = 0

    procs = [Process_shared(var) for i in range(3)]

    for p in procs:
        p.start()

    for p in procs:
        p.join()

    print(var.value)

Se prueba el resultado

In [ ]:
test()

Como se puede ver, el resultado no es necesariamente 30.000, esto se debe al acceso simultaneo y aleatorio de los procesos a `var`, para solucionar este problema se hace uso de `lock`, para ello se redefine la clase `Process_shared` observando que lock es un *context manager*

In [ ]:
class Process_shared_lock(Process):
    def __init__(self, var, n=10000):
        super().__init__()
        self.var = var
        self.n = n

    def run(self):
        for i in range(self.n):
            with lock:
                self.var.value += 1

Se redefine la prueba asociada y se ejecuta:

In [ ]:
def test():
    var = Value("i")
    var.value = 0

    procs = [Process_shared_lock(var) for i in range(3)]

    [p.start() for p in procs]
    [p.join() for p in procs]

    print(var.value)


test()

Con lo cual se obtiene el resultado buscado.

Sin embargo, coordinar procesos más complejos se torna tedioso y complejo, además de ser susceptible a errores.
Por lo general, se recomienda, a menos que sea estrictamente necesario, a librerías que facilitan la paralelización, como las que vamos a ver a continuación.

---

### Paralelización con `Joblib`



<div align='center'>
    <img src='https://raw.githubusercontent.com/MDS7202/MDS7202/main/recursos/2023-01/23_compilacion_y_paralelismo/joblib.png' width=200>
</div>

Otra forma de paralelizar de forma relativamente sencilla es usar la librería `joblib`. 
Esta permite ejecutar funciones de forma paralela, pero ahora de manera funcional. 
Es decir, le entregamos una función y una lista de argumentos y ejecuta una función con dichos argumentos de forma paralela.

Para esto, utiliza el decorador `delayed` sobre una función (lo que la transforma a lazy, es decir, no se ejecuta instantaneamente). Luego a través del objeto `Parallel` que toma el número de trabajos concurrentes que se ejecutarán (`n_jobs`) ejecuta las funciones con sus parámetros.

El siguiente ejemplo veremos como paralelizar el cálculo de coseno sobre una lista:

In [ ]:
import numpy as np

[np.cos(i) for i in np.arange(0, 1, 0.1)]

La notación es muy similar a un list comprehension, solo que con 2 diferencias:

- Se reemplazan los corchetes exteriores `[f(i) for i in ...]` por paréntesis `(f(i) for i in ...)` (Esto da lugar a un generador en vez de una lista).
- Se encapsula la función a aplicar a cada elemento con la función `delayed`, o sea, `f(i)` por `delayed(f)(i)`.

Luego, lo anterior se le pasa como argumento a un Parallel.


In [ ]:
from math import sqrt
import numpy as np

from joblib import Parallel, delayed

# n_jobs=-1 indica que se usarán todos los procesadores disponibles.
Parallel(n_jobs=-1)(delayed(np.cos)(i) for i in np.arange(0, 1, 0.1))

In [ ]:
%timeit [np.cos(i) for i in np.arange(0, 1, 0.001)]

In [ ]:
%timeit Parallel(n_jobs=-1)(delayed(np.cos)(i) for i in np.arange(0, 1, 0.001))

Para tareas numéricas no es tan efectivo (ya existe un cierto overhead/coste de generar los subprocesos), pero para tareas pesadas, se comporta bastante bien.

Para este ejemplo, leeremos 50 veces archivo con números aleatorios en forma secuencial y en forma paralelizada:

In [ ]:
import pandas as pd


def leer_archivo(_):
    _ = pd.read_csv("https://raw.githubusercontent.com/MDS7202/MDS7202/main/recursos/2023-01/23_compilacion_y_paralelismo//num_aleatorios.csv")

In [ ]:
%timeit [leer_archivo(_) for _ in range(0, 10)]

In [ ]:
%timeit Parallel(n_jobs=-1)(delayed(leer_archivo)(_) for _ in range(0, 10))

Ahora si notamos diferencias.

### Asincronía y Corrutinas

<center>
<img src='https://raw.githubusercontent.com/MDS7202/MDS7202/main/recursos/2023-01/23_compilacion_y_paralelismo/corrutinas.png' />
</center>

En general, se utiliza más en el desarrollo web/software para no bloquear la ejecución de código al solicitar datos a un servidor externo o ejecutar un proceso muy pesado.


> **Pregunta ❓**: ¿Qué aplicación de data science podría beneficiarse de la asincronía?

## Procesamiento Distribuido

<img src='https://raw.githubusercontent.com/MDS7202/MDS7202/main/recursos/2023-01/23_compilacion_y_paralelismo/distributed.png'/>

El procesamiento distribuido hace referencia a la ejecución de tareas utilizando múltiples máquinas. Por lo general se refiere al trabajo con clusters de procesamiento y suele llevarse a cabo por medio de herramientas como [`Spark`](https://spark.apache.org/) o [`Dask`](https://www.dask.org/).

Diferencias entre spark y dask: https://docs.dask.org/en/stable/spark.html

En esta última sección estudiaremos `Dask` para procesar `DataFrames`.



### `Dask`
<div align='center'>
<img src='https://raw.githubusercontent.com/MDS7202/MDS7202/main/recursos/2023-01/23_compilacion_y_paralelismo/dask.jpg' width=300>
</div>

Dask permite escalar procesos de Python (ya sea en un computador personal o un cluster) de manera sencilla. Provee de funcionalidades para tratar, por medio de procesamiento multi-core, con datsets masivos **que por lo general no caben en memoria.**

`Dask` fue implementado como un reemplazo de `Numpy` y `Pandas`, por ende su interfaz de usuario (API) es muy similar.
Los `DataFrames` de Dask son en términos prácticos conjuntos de `DataFrames` de pandas. Dicha separación permite ejecutar operaciones distribuidas y paralelas de forma muy eficiente.



<div align='center'>
<img src='https://raw.githubusercontent.com/MDS7202/MDS7202/main/recursos/2023-01/23_compilacion_y_paralelismo/dask.png' width=600 />
</div>

<center>
<img src='https://raw.githubusercontent.com/MDS7202/MDS7202/main/recursos/2023-01/23_compilacion_y_paralelismo/dask_mimic.png' width=700>
</center>

Pueden encontrar mayor información en la página oficial del proyecto:

https://docs.dask.org/en/latest/

In [ ]:
!python -m pip install "dask[dataframe]"

#### Pandas vs Dask

##### Datos aleatorios con `pd.DataFrame`

In [ ]:
import numpy as np
import pandas as pd

# generamos datos aleatorios (disminuir en caso de no contar con suficiente memoria)
df = pd.DataFrame(np.random.random((2000000,200)))

# generamos categorías a partir de bins para luego agregar
df[0] = pd.cut(df[0], 20)
df[1] = pd.cut(df[1], 20)
df[2] = pd.cut(df[2], 20)

df.info()

La prueba será cuanto se demora en ejecutar un `groupby(..).mean()` sobre las categorías generadas:

In [ ]:
df.groupby([0, 1, 2]).mean()

##### Inicializar `dask.dataframe`

Ahora, generamos un `Dask DataFrame` desde pandas

In [ ]:
import dask.dataframe as dd

ddf = dd.from_pandas(df, npartitions=5)
ddf.info()

Vemamos que pasa si hacemos la misma operación que antes:

In [ ]:
ddf.groupby([0, 1, 2]).mean()

No produjo ningún resultados. Esto es porque Dask es **Lazy**, es decir, se ejecuta solo cuando alguien demanda su ejecución.
Esto se puede lograr a partir del método `compute()`:


In [ ]:
ddf.groupby([0, 1, 2]).mean().compute()

Incluso se puede visualizar como se computa la operación distribuida a través de el siguiente método:


In [ ]:
ddf.groupby([0, 1, 2]).mean().visualize()

##### Comparación de tiempos

En las siguientes celdas ejecutamos al comparación de tiempos

In [ ]:
%timeit df.groupby([0, 1, 2]).mean()

In [ ]:
%timeit ddf.groupby([0, 1, 2]).mean().compute()

Podemos ver que **Dask** no es más rápido que **pandas** para la cantidad de datos anterior `(500000 x 200) ~ 752.9 MB`.

Nuevamente, esto se debe al overhead / gasto adicional que implica lanzar varios procesos para ejecutar tareas en paralelo.

## Polars
<div align="center">
<img src="https://raw.githubusercontent.com/pola-rs/polars-static/master/logos/polars_github_logo_rect_dark_name.svg" width=450>
</div>

<div align="center">
Blazingly Fast DataFrame Library 
</div>

Nueva librería alternativa a pandas enfocada en alto rendimiento y programado íntegramente en [Rust](https://www.rust-lang.org/es).

Sus principios son:

- **Rápido**: Polars está escrito desde cero, diseñado cerca de la máquina y sin dependencias externas.
- **E/S**: Soporte para todas las capas comunes de almacenamiento de datos: local, almacenamiento en la nube y bases de datos.
- **Fácil de usar**: Escriba sus consultas de la forma en que fueron concebidas. Polars, internamente, determinará la forma más eficiente de ejecutar utilizando su optimizador de consultas.
- **_Out of core_**: Polars soporta la transformación de datos fuera del núcleo con su API de streaming. Permitiéndole procesar sus resultados sin requerir que todos sus datos estén en memoria al mismo tiempo.
- **Paralelo**: Polars utiliza plenamente la potencia de su máquina dividiendo la carga de trabajo entre los núcleos de CPU disponibles sin ninguna configuración adicional.
- **Motor de consulta vectorizado**: Polars utiliza Apache Arrow, un formato de datos en columnas, para procesar sus consultas de forma vectorizada.

In [ ]:
!pip install polars[all]

In [ ]:
import polars as pl

pl_df = pd.DataFrame(df)

In [ ]:
%timeit df.groupby([0, 1, 2]).mean()

In [ ]:
%timeit pl_df.groupby([0, 1, 2]).mean()

### Debo usar estos frameworks?

Si tu dataset cabe en memoria comodamente y no es muy grande, entonces no es necesario usar `Dask`. Simplemente agregará una capa de complejidad al desarrollo.

Por otra parte, si el dataset con el cuál están trabajando es masivo, entonces dichas librerías son un buen factor a considerar.

## Y para usar la GPU?

Pueden utilizar estas alternativas:

- [Cupy](https://docs.cupy.dev/en/stable/user_guide/basic.html) - Implementación de `NumPy/SciPy` usando `CUDA` (para GPUs nvidia)
- [Rapids](https://rapids.ai/start.html): Colección de librerías basadas en `CUDA` que al igual que la alternativa anterior, mejora el rendimiento a través de la GPU.